In [1]:
import numpy as np
import torch
import pickle as pkl
import ipycytoscape
import networkx as nx

def vis(G):
    cso = ipycytoscape.CytoscapeWidget()
    cso.graph.add_graph_from_networkx(G)
    cso.set_style([
                            {
                                'selector': 'node',
                                'css': {
                                    'background-color': 'red',
                                    'content': 'data(node_label)' #
                                }
                            },
                                                    {
                                'selector': 'edge',
                                'css': {
                                    'content': 'data(edge_label)' #
                                }
                            }
                
                ])

    for i in range(len(cso.graph.nodes)):
        id = int(cso.graph.nodes[i].data['id'])
        label = cso.graph.nodes[i].data['node_label']
        new_label = f"{id}: {label}"
        cso.graph.nodes[i].data['node_label'] = new_label


    # for i in range(len(cso.graph.edges)):
    #     label = cso.graph.edges[i].data['edge_label']
    #     new_label = f"{label}"
    #     cso.graph.edges[i].data['edge_label'] = new_label

    return cso
    
# Test it with output graph
import pickle
#with open('datasets/DD/data.pkl','rb') as f:
with open('../datasets/ZINC_TEST/data.pkl','rb') as f:
    data = pickle.load(f)
out = vis(data[3])
display(out)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'css': {'background-c…

In [2]:

class SparseGraph:
    # Convert a graph to a sparse representation (numpy matrices)
    def __init__(self, G):
        # Convert a networkx graph (with edge and node labels) to a sparse graph format

        # Edge index Matrix
        idxs = np.array(G.edges).transpose() # (2,|E|) dim. array idxs[:,j] = [u,v]^T indicates endpoints of j'th edge e=u->v
        idxs = np.concatenate((idxs, idxs[[1,0]]), axis=1) # idxs[[1,0]] flips the two rows ie [u,v]^T -> [v,u]^T, so by concat now have (2, 2*|E|)
        self.idxs = torch.from_numpy(idxs) #.astype(np.float32))

        # Node features
        Xv = np.array([G.nodes[idx]['node_label'] for idx in G.nodes]).transpose()#.reshape(-1,1) # Node feature matrix of dim (reshape: (|V|,) -> (|V|,1))
        self.Xv = torch.from_numpy(Xv.astype(np.float32))
        
        # Edges features
        Xe = np.array([G.edges[idx]['edge_label'] for idx in G.edges]).transpose()#.reshape(-1,1) # Edge feature matrix of dim (reshape: (|E|,) -> (|E|,1))
        Xe = np.concatenate((Xe,Xe), axis=0)
        self.Xe = torch.from_numpy(Xe.astype(np.float32))

        # Get Graph features
        y = G.graph['label']
        self.y = torch.from_numpy(y.astype(np.float32))

    def to_nx(self):
        # Convert the sparse graph back to a networkx gaph g

        # Convert tensors to numpy
        idxs = self.idxs.numpy().astype('int')
        Xv = self.Xv.numpy()
        Xe = self.Xe.numpy()

        g = nx.Graph() # Empty nx graph

        # Add edges (nodes added automatically)
        for j in range(idxs.shape[1]):
            g.add_edge(idxs[0,j], idxs[1,j])
        
        # Set Node and Edge Weights
        nx.set_node_attributes(g, {idx: Xv[idx] for idx in range(Xv.shape[0])}, "node_label")
        nx.set_edge_attributes(g, {(idxs[0,idx], idxs[1,idx]): Xe[idx] for idx in range(int(Xe.shape[0]/2))}, "edge_label")

        # TODO: Convert graph label in networkx
        return g


class MyDataset(torch.utils.data.Dataset):
    def __init__(self, nx_graph_list):
        self.np_sparse_graphs = [SparseGraph(g) for g in nx_graph_list]

    def __len__(self):
        return len(self.np_sparse_graphs)
    
    def __getitem__(self, idx):
        return self.np_sparse_graphs[idx]
        #return torch.from_numpy(sg.idxs), torch.from_numpy(sg.Xv), torch.from_numpy(sg.Xe), torch.from_numpy(sg.Xe)


SG = SparseGraph(data[3])
G1 = SG.to_nx()
vis(G1)

def MyCollate(sparse_graph_list):
    #sparse_graph_list = [SparseGraph(data[0]), SparseGraph(data[1]), SparseGraph(data[2]) ]
    #sgl = sparse_graph_list

    # Create empty SparseGraph Object (avoid calling init, we will initialize here alreadt)
    output = SparseGraph.__new__(SparseGraph)

    # By joining graphs, the node indexes need to she shifted
    # Ie if the first graph has 10 nodes, then for the second graph the node indexes 0,1,2,... --> 10,11,12,...

    # compute batch_idx matrix, and a lookup table for how much to shift each graph's nodes indexes by
    node_idx_shift = [0] # Lookup table for the node index shift of each graph
    output.batch_idx = []
    tot_num_nodes = 0 # Total number of nodes
    for i,sg in enumerate(sparse_graph_list):
        num_nodes = sg.Xv.shape[0]
        tot_num_nodes += num_nodes
        node_idx_shift.append(tot_num_nodes)
        output.batch_idx += [i]*num_nodes

    # First shift all the node indexes in each graph, and concatenate them
    output.idxs = torch.cat([sg.idxs + torch.from_numpy(np.array([node_idx_shift[i], node_idx_shift[i]]).transpose().reshape(-1,1))  # idxs + [idx_shift, idx_shift]^T
                            for i, sg in enumerate(sparse_graph_list)],
                        dim = 1)

    # Concatenate Node and Edge feature vectors, and graph labels
    output.Xv = torch.cat([sg.Xv  for sg in sparse_graph_list])
    output.Xe = torch.cat([sg.Xe for sg in sparse_graph_list])
    output.y = torch.cat([sg.y for sg in sparse_graph_list])

    return output

sgl = [SparseGraph(data[0]), SparseGraph(data[1])] #SparseGraph(data[2]) ]
res = MyCollate(sgl)
vis(res.to_nx())


CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'css': {'background-c…

In [25]:

from torch import nn
class SLP(torch.nn.Module):
    # Single Layer Perceptron (Dummy Function)
    def __init__(self, in_features, out_features):
        super(SLP, self).__init__()
        self.fc = nn.Linear( in_features, out_features)
        self.relu = torch.nn.ReLU() # instead of Heaviside step fn
        
    def forward(self, x):
        x = self.fc(x)
        x = self.relu(x) # instead of Heaviside step fn
        return x

U = SLP(2,1)
x = torch.tensor(np.array([1,1]), dtype=torch.float32) #.reshape(-1,1),
U.forward(x)

tensor([0.1088], grad_fn=<ReluBackward0>)

In [106]:
#x = torch.from_numpy(np.array([1,1]).transpose().reshape(-1,1))
sg = sgl[0]
num_nodes = sg.Xv.shape[0]
H = torch.ones((num_nodes))

U = SLP(2,1)
X = torch.transpose(torch.stack([H[sg.idxs[0,:]], sg.Xe]),0,1)
Y = U.forward(X)
Y.shape
Y = torch.ones(Y.shape)

Z = torch.zeros((2*num_nodes, 1), dtype=torch.float32)
Z = Z.scatter_add_(dim=0, index= sg.idxs[1,:].reshape(-1,1), src=Y)
Z

tensor([[1.],
        [3.],
        [2.],
        [2.],
        [3.],
        [2.],
        [2.],
        [3.],
        [2.],
        [2.],
        [2.],
        [2.],
        [3.],
        [1.],
        [2.],
        [2.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])

In [109]:
sg.idxs[2,:].shape

IndexError: index 2 is out of bounds for dimension 0 with size 2

In [99]:
sg.idxs[1,:].reshape(-1,1).shape

torch.Size([34, 1])

In [39]:
torch.Tensor.scatter(0, sg.idxs[1,:], Y)

TypeError: descriptor 'scatter' for 'torch._C._TensorBase' objects doesn't apply to a 'int' object

In [87]:
Z = torch.empty((16,1), dtype=torch.float32)
#idxx = torch.from_numpy(np.array([0]*17+[1]*17).astype('int64').reshape(-1,1))
idxx = (sg.idxs[1,:]).reshape(-1,1)

Z = Z.scatter_add(dim=0, index=idxx, src=Y)

In [86]:
idxx = (sg.idxs[1,:]).reshape(-1,1)
idxx.shape

torch.Size([34, 1])

torch.Size([34, 1])

In [77]:
Y.shape

torch.Size([34, 1])

In [63]:
idxx = torch.from_numpy(np.array([0]*8+[1]*8))

tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.int32)